# Maximum Error in CLIRAD-LW's Cooling Rate Estimates

The maximum is over the layers in either the troposphere or the stratosphere and over the mls, saw and trp atmosphere profiles.  The values are the absolute values.

In [141]:
%reload_ext autoreload
%autoreload 2

import pymongo
import pandas as pd

from climatools.cliradlw.utils import *
from climatools.atm.absorbers import *
from climatools.atm.regions import *
from climatools.dataset import *

import pprint
from IPython import display

In [33]:
COMMITNUMBER = '1013f91'

In [34]:
client = pymongo.MongoClient('localhost', 27017)

In [35]:
MOLECULES = ('h2o', 'co2', 'o3', 'n2o', 'ch4')
BANDS = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)
ATMPROS = dict([('mls', 294), ('saw', 257), ('trp', 300)])

In [142]:
def get_clirad_subparams(molecule=None, band=None):
    '''
    Return mls, saw and trp clirad parameter sub-dictionaries
    for a given molecule and clirad-lw spectral band.
    '''
    conc = nongreys_byband()[band][molecule]
    return {atmpro: {'commitnumber': COMMITNUMBER,
                     'band': [band],
                     'molecule': {molecule: conc},
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            for atmpro, tsfc in ATMPROS.items()}

def to_lblnewparam(param=None):
    '''
    Convert clirad's input parameter dictionary 
    into something like lblnew's.
    '''
    band = param['band'][0]
    band = mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    
    p = {'band': band, 
         'atmpro': atmpro, 'tsfc': tsfc,
         'dv': .001, 'nv': 1000}
    
    if len(param['molecule']) > 1: p['molecule'] = param['molecule']
    else: 
        molecule, conc = list(param['molecule'].items())[0]
        p['molecule'] = molecule
        if conc == 'atmpro': p['conc'] = None
        else: p['conc'] = conc
    return p

def make_query(param=None):
    molecules = set(MOLECULES)
    q = {}
    for n, v in param.items():
        if n == 'molecule' and type(v) == dict:
            for mol, conc in v.items():
                q[f'param.molecule.{mol}'] = conc
            for mol in molecules - set(v.keys()):
                q[f'param.molecule.{mol}'] = {'$exists': 0}
        else:
            q[f'param.{n}'] = v
    return q

In [182]:
def get_maxerr(molecule=None, band=None):
    '''
    Returns the maximum deviation from crd at the top and bottom
    of the atmosphere.
    '''
    tropos = tropopause_pressures()
    
    dclis = {atmpro: client.cliradnew.lw.find_one(make_query(param=p)) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dclis = {atmpro: CliradnewLWModelData.from_mongodoc(doc) for atmpro, doc in dclis.items()}
    dclis = {atmpro: d.wgt_cool.sel(i=1).sum('band') for atmpro, d in dclis.items()}

    dcrds = {atmpro: client.lblnew.bestfit_lw.find_one(make_query(param=to_lblnewparam(p))) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dcrds = {atmpro: LBLnewBestfitModelData.from_mongodoc(doc) for atmpro, doc in dcrds.items()}
    dcrds = {atmpro: d.crd_cool.sum('g') for atmpro, d in dcrds.items()}
    
    ddifs = {atmpro: dcli - dcrds[atmpro] for atmpro, dcli in dclis.items()}

    ddifs_tropo = [abs(d.where(d.pressure > tropos[atmpro])).max() for atmpro, d in ddifs.items()]
    ddifs_strat = [abs(d.where(d.pressure < tropos[atmpro])).max() for atmpro, d in ddifs.items()]
    
    ddifs_tropo = [float(d['coolrg'].values) for d in ddifs_tropo]
    ddifs_strat = [float(d['coolrg'].values) for d in ddifs_strat]
    
    return dict(troposphere=max(ddifs_tropo), stratosphere=max(ddifs_strat))


In [183]:
def maxerr_all():
    df = pd.DataFrame(index=MOLECULES, 
                      columns=pd.MultiIndex.from_product([BANDS, ('troposphere', 'stratosphere')], 
                                                         names=('band', 'region')))
    df.index.name = 'gas'

    for band, molconcs in nongreys_byband().items():
        for mol, conc in molconcs.items():
            for level, v in get_maxerr(molecule=mol, band=band).items():
                df.loc[mol, (band, level)] = round(v, 5)

    df.fillna('', inplace=True)
    return df

In [184]:
def script():
    df = maxerr_all()
    display.display(df.transpose())

In [185]:
script()

gas                    h2o      co2       o3      n2o      ch4
band region                                                   
1    troposphere   0.05664                                    
     stratosphere  2.60631                                    
2    troposphere   0.0848                                     
     stratosphere  0.08979                                    
3    troposphere   0.04771  0.0082            0.00142         
     stratosphere  0.00199  0.17273           0.00478         
4    troposphere   0.05037  0.0183                            
     stratosphere  0.00257  22.2754                           
5    troposphere   0.02228  0.01074                           
     stratosphere  0.00079  0.17034                           
6    troposphere   0.04972  0.00051                           
     stratosphere  0.00017  0.00574                           
7    troposphere   0.01596  0.00031  0.00893                  
     stratosphere  9e-05    0.00622  0.18143                  
8    troposphere   0.0248                                     
     stratosphere  0.00055                                    
9    troposphere   0.02617                    0.01056  0.00407
     stratosphere  0.00751                    0.02487  0.04086
10   troposphere   0.02929                                    
     stratosphere  0.04125                                    
11   troposphere   0.01406  0.00302                           
     stratosphere  0.00147  0.15067

In [181]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')